In [157]:
import numpy as np
import pandas as pd
from math import sqrt
from scipy import optimize as op

# Data Preprocessing

In [158]:
main_data = pd.read_csv('./04_cricket_1999to2011.csv')

data = main_data[main_data['Innings']==1]
data = data[['Over','Runs','Total.Out']]
# Nx50 Matrix holding history data 
Innings1 = list()


inning = np.zeros(100)
i=0
for d in data.itertuples():
    if d[1]==1 or i==50:
        while i!=50:
            inning[i],inning[i+50] = inning[i-1],inning[50+i-1]
            i+=1
        Innings1.append(inning)
        inning = np.zeros(100)
        i=0
    inning[i],inning[i+50] = inning[i-1]+d[2],d[3]
    i+=1

Innings1 = np.array(Innings1)     
Innings1 = Innings1[1:,:]


## mRSC Algorithm

In [159]:
def mRSC(X,data,intervention_point,k):

    U,S,V = np.linalg.svd(data)
    S_ = np.diag(S[:k])
    M = U[:,:k]@S_@V[:k,:]    

    M_k = np.zeros(shape=(M.shape[0],2*intervention_point))
    M_k[:,:intervention_point],M_k[:,intervention_point:] = M[:,:intervention_point],M[:,50:50+intervention_point] 

    opt_result = op.least_squares(lambda v : X-v@M_k,np.zeros(M_k.shape[0]))
    xopt = opt_result.x
    result = np.int16(np.round(xopt@M))
    return result

## Task 1 : Estimate Deviations in Predictions of 1st Innings

In [160]:
np.random.shuffle(Innings1)
P = int(Innings1.shape[0]*0.9)
train,test = Innings1[:P,:],Innings1[P:,:]

In [164]:
error = np.zeros(50)
for inning in test[np.random.randint(0,test.shape[0],20),:]:
    for intervention_point in range(50):
        Z = np.zeros(2*intervention_point)
        Z[:intervention_point],Z[intervention_point:] = inning[:intervention_point],inning[50:50+intervention_point]
        prediction = mRSC(Z,train,intervention_point,10)
        error[intervention_point] += sqrt(np.mean(np.square(inning-prediction)))
error /= 20
print(error)

[9.81109467e+01 6.23934689e+01 2.23352346e+02 2.95436954e+02
 3.14614932e+02 2.30069820e+03 5.91751535e+02 6.15283888e+02
 6.01254699e+02 4.51087512e+02 3.09131354e+02 3.09298930e+02
 2.52392260e+02 2.31022187e+02 2.39297301e+02 1.81801764e+02
 1.75749843e+02 1.47397813e+02 1.36368752e+02 1.27365231e+02
 1.20439044e+02 1.12228827e+02 1.06979433e+02 8.14631265e+01
 7.35946044e+01 7.45518704e+01 7.09066128e+01 6.96139882e+01
 6.43192768e+01 5.20095116e+01 4.80301418e+01 4.55997131e+01
 4.05401142e+01 3.25424447e+01 2.62868450e+01 2.54435307e+01
 2.54388985e+01 2.09333364e+01 1.65341346e+01 1.88895494e+01
 1.74677000e+01 1.33557682e+01 1.25369528e+01 1.26652611e+01
 9.35231487e+00 8.19370045e+00 4.50481523e+00 3.04358041e+00
 2.16814464e+00 1.90596803e+00]
